In [14]:
from GPT import *


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

valid_df = build_valid_df(CSV_PATH, IMG_ROOT)
if valid_df.empty:
    print("[WARN] No valid rows found; check paths and PNG conversion.")

labels_as_str = valid_df[TEXT_COL].astype(str).tolist()
tokenizer = build_tokenizer_from_labels(labels_as_str)
pad_id = getattr(tokenizer, "pad_token_id", 0)
bos_id = getattr(tokenizer, "bos_token_id", 1)
eos_id = getattr(tokenizer, "eos_token_id", 2)

# DINO expects 224 or 518 square; 224 is fine here
IMG_SIZE = 1024
tf = dino_image_transform(img_size=IMG_SIZE)
ds = CheXpertDataset(img_root=IMG_ROOT, csv=valid_df, transform=tf, text_col=TEXT_COL)
collate_fn = CaptionCollate(tokenizer, pad_id)

is_windows = os.name == "nt"
num_workers = 0 if is_windows else 2
persistent_workers = False if num_workers == 0 else True

# Full loader (used to sample subsets below)
full_loader = DataLoader(
    ds,
    batch_size=8,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    persistent_workers=persistent_workers,
    collate_fn=collate_fn
)

# Simple split
n_total = len(ds)
n_train = int(n_total * 0.8)
n_valid = int(n_total * 0.9)
indices = torch.randperm(n_total).tolist()
train_idx, valid_idx, test_idx = indices[:n_train], indices[n_train:n_valid], indices[n_valid:]
train_ds = torch.utils.data.Subset(ds, train_idx)
valid_ds = torch.utils.data.Subset(ds, valid_idx)
test_ds = torch.utils.data.Subset(ds, test_idx)
train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, collate_fn=collate_fn, num_workers=num_workers)
valid_loader = DataLoader(valid_ds, batch_size=8, shuffle=False, collate_fn=collate_fn, num_workers=num_workers)
test_loader = DataLoader(test_ds, batch_size=8, shuffle=False, collate_fn=collate_fn, num_workers=num_workers)

# DINO ViT-S/16 hidden size is 384 (for this checkpoint); adjust if you change encoder
D_IMG = 384
N_PREFIX = 1 #(IMG_SIZE // 16) ** 2  # number of visual prefix tokens (including CLS)
model = DinoGPTCaptioner(
    vocab_size=tokenizer.vocab_size,
    d_img=D_IMG,
    pad_id=pad_id,
    d_model=512,
    n_layer=8,
    n_head=8,
    n_prefix=N_PREFIX,           # number of visual prefix tokens
    max_seq_len=256,
    dino_model_id="facebook/dinov3-vits16-pretrain-lvd1689m",
    freeze_dino=True,
).to(device)

optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()), lr=3e-4, weight_decay=1e-2
)


Using device: cuda
[INFO] Kept 47494/223462 rows with existing PNGs under C:\Users\emman\Desktop\PROYECTOS_VS_CODE\PRUEBAS_DE_PYTHON\CheXpertPlus\PNG


In [ ]:
# ---- Train a few slices just to validate wiring ----
for epoch in range(50):
    slice_train_loader = islice(train_loader, 10)
    slice_valid_loader = islice(valid_loader, 10)
    train_stats = train_one_epoch(model, slice_train_loader, optimizer, device, pad_id, num_batches=10, grad_clip=1.0)
    val_stats = evaluate(model, slice_valid_loader, device, pad_id, num_batches=10)
    print(f"Epoch {epoch + 1}: Train Loss={train_stats['loss']:.4f}, PPL={train_stats['ppl']:.2f} | "
            f"Val Loss={val_stats['val_loss']:.4f}, Val PPL={val_stats['val_ppl']:.2f}")

def sequence_ce_loss(logits, labels, pad_id):
    """
    logits: (B, T, V) — corresponds to input_ids[:, :] positions
    labels: (B, T) — next tokens; pad ignored
    """
    B, T, V = logits.size()
    loss_fn = nn.CrossEntropyLoss(ignore_index=pad_id, label_smoothing=0.1)
    return loss_fn(logits.reshape(B * T, V), labels.reshape(B * T))

@torch.no_grad()
def batch_perplexity(logits, labels, pad_id):
    loss = sequence_ce_loss(logits, labels, pad_id)
    return float(math.exp(min(loss.item(), 20.0)))

slice_test_loader = islice(test_loader, 1)
test_stats = evaluate(model, slice_test_loader, device, pad_id, num_batches=1)
print(f"Test Loss={test_stats['val_loss']:.4f}, Test PPL={test_stats['val_ppl']:.2f}")
# ---- Quick generation sanity check ----
with torch.no_grad():
    for pixel_values, ids_loader, paths, raw_labels in test_loader:
        pixel_values = pixel_values.to(device)
        gen_ids = model.generate(
            pixel_values=pixel_values,
            bos_id=bos_id, eos_id=eos_id,
            max_new_tokens=256, top_p=0.9, temperature=0.9, greedy=True
        )
        print("Predictions (first batch):")
        for i in range(min(gen_ids.size(0), 8)):
            print(f"\nGEN {i+1}:", tokenizer.decode(gen_ids[i].tolist()))
            print(f"TGT {i+1}:", tokenizer.decode(ids_loader[i].tolist()))
            # Calculate loss between generated and target sequences
        del pixel_values, ids_loader, paths, raw_labels, gen_ids
        torch.cuda.empty_cache()
        break

Evaluating: 100%|██████████| 10/10 [00:09<00:00,  1.01it/s]


Epoch 1: Train Loss=5.0197, PPL=158.71 | Val Loss=4.9587, Val PPL=145.70


Evaluating: 100%|██████████| 10/10 [00:09<00:00,  1.03it/s]


Epoch 2: Train Loss=4.8745, PPL=135.51 | Val Loss=4.7260, Val PPL=115.78


Evaluating: 100%|██████████| 10/10 [00:10<00:00,  1.01s/it]


...

Epoch 49: Train Loss=3.2541, PPL=26.22 | Val Loss=3.2966, Val PPL=27.47


Evaluating: 100%|██████████| 10/10 [00:10<00:00,  1.03s/it]


Epoch 50: Train Loss=3.2785, PPL=27.09 | Val Loss=3.2904, Val PPL=27.27


Evaluating: 100%|██████████| 1/1 [00:01<00:00,  1.00s/it]

Test Loss=3.3161, Test PPL=27.55


In [24]:
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()), lr=5e-4, weight_decay=1e-2
)

# ---- Train a few slices just to validate wiring ----
for epoch in range(50):
    slice_train_loader = islice(train_loader, 10)
    slice_valid_loader = islice(valid_loader, 10)
    train_stats = train_one_epoch(model, slice_train_loader, optimizer, device, pad_id, num_batches=10, grad_clip=1.0)
    val_stats = evaluate(model, slice_valid_loader, device, pad_id, num_batches=10)
    print(f"Epoch {epoch + 1}: Train Loss={train_stats['loss']:.4f}, PPL={train_stats['ppl']:.2f} | "
            f"Val Loss={val_stats['val_loss']:.4f}, Val PPL={val_stats['val_ppl']:.2f}")

def sequence_ce_loss(logits, labels, pad_id):
    """
    logits: (B, T, V) — corresponds to input_ids[:, :] positions
    labels: (B, T) — next tokens; pad ignored
    """
    B, T, V = logits.size()
    loss_fn = nn.CrossEntropyLoss(ignore_index=pad_id, label_smoothing=0.1)
    return loss_fn(logits.reshape(B * T, V), labels.reshape(B * T))

@torch.no_grad()
def batch_perplexity(logits, labels, pad_id):
    loss = sequence_ce_loss(logits, labels, pad_id)
    return float(math.exp(min(loss.item(), 20.0)))

slice_test_loader = islice(test_loader, 1)
test_stats = evaluate(model, slice_test_loader, device, pad_id, num_batches=1)
print(f"Test Loss={test_stats['val_loss']:.4f}, Test PPL={test_stats['val_ppl']:.2f}")
# ---- Quick generation sanity check ----
with torch.no_grad():
    for pixel_values, ids_loader, paths, raw_labels in test_loader:
        pixel_values = pixel_values.to(device)
        gen_ids = model.generate(
            pixel_values=pixel_values,
            bos_id=bos_id, eos_id=eos_id,
            max_new_tokens=256, top_p=0.9, temperature=0.9, greedy=True
        )
        print("Predictions (first batch):")
        for i in range(min(gen_ids.size(0), 8)):
            print(f"\nGEN {i+1}:", tokenizer.decode(gen_ids[i].tolist()))
            print(f"TGT {i+1}:", tokenizer.decode(ids_loader[i].tolist()))
            # Calculate loss between generated and target sequences
        del pixel_values, ids_loader, paths, raw_labels, gen_ids
        torch.cuda.empty_cache()
        break

Evaluating: 100%|██████████| 10/10 [00:10<00:00,  1.01s/it]


Epoch 1: Train Loss=3.5706, PPL=36.86 | Val Loss=3.4515, Val PPL=32.04


Evaluating: 100%|██████████| 10/10 [00:09<00:00,  1.02it/s]


Epoch 2: Train Loss=3.4131, PPL=31.49 | Val Loss=3.4323, Val PPL=31.41


Evaluating: 100%|██████████| 10/10 [00:10<00:00,  1.00s/it]


...

Epoch 49: Train Loss=3.0437, PPL=21.32 | Val Loss=3.1379, Val PPL=23.33


Evaluating: 100%|██████████| 10/10 [00:10<00:00,  1.04s/it]


Epoch 50: Train Loss=3.0657, PPL=21.93 | Val Loss=3.1443, Val PPL=23.47


Evaluating: 100%|██████████| 1/1 [00:00<00:00,  1.01it/s]


Test Loss=3.0535, Test PPL=21.19
Predictions (first batch):

GEN 1: 1. pa and lateral views of the chest demonstrate stable positioning of a right upper extremity picc line. 2. no evidence of acute cardiopulmonary disease. 3. no significant interval change in cardiomegaly. degenerative changes of the thoracic spine. osteopenia.., no acute osteopenia., no acute osteopenia, with, pleural effusion. ). - 2008 hours demonstrates no acute osteopenia, pulmonary edema., unchanged. - 2008. osteopenia, osteopenia, unchanged. osteopenia, pleural effusion. )., unchanged., unchanged. osteopenia, no acute osteopenia, no significant osteopenia, no acute osteopenia, right - 6 - 6 - 6 - 6 - 2008., md at 10 - 2008 at the thoracic spine. - 6 - 2008. - 2008, no acute osteopenia, no significant osteopenia, no significant interval change in the attending radiographic change.,
TGT 1: 1. chest 2 views, demonstrate no focal consolidation or pleural effusion. stable overall aeration and volume 2. cardiac silhou

In [25]:
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()), lr=2e-4, weight_decay=1e-2
)

lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

# ---- Train a few slices just to validate wiring ----
for epoch in range(50):
    slice_train_loader = islice(train_loader, 10)
    slice_valid_loader = islice(valid_loader, 10)
    train_stats = train_one_epoch(model, slice_train_loader, optimizer, device, pad_id, num_batches=10, grad_clip=1.0)
    val_stats = evaluate(model, slice_valid_loader, device, pad_id, num_batches=10)
    lr_scheduler.step()
    print(f"Epoch {epoch + 1}: Train Loss={train_stats['loss']:.4f}, PPL={train_stats['ppl']:.2f} | "
            f"Val Loss={val_stats['val_loss']:.4f}, Val PPL={val_stats['val_ppl']:.2f}")

def sequence_ce_loss(logits, labels, pad_id):
    """
    logits: (B, T, V) — corresponds to input_ids[:, :] positions
    labels: (B, T) — next tokens; pad ignored
    """
    B, T, V = logits.size()
    loss_fn = nn.CrossEntropyLoss(ignore_index=pad_id, label_smoothing=0.1)
    return loss_fn(logits.reshape(B * T, V), labels.reshape(B * T))

@torch.no_grad()
def batch_perplexity(logits, labels, pad_id):
    loss = sequence_ce_loss(logits, labels, pad_id)
    return float(math.exp(min(loss.item(), 20.0)))

slice_test_loader = islice(test_loader, 1)
test_stats = evaluate(model, slice_test_loader, device, pad_id, num_batches=1)
print(f"Test Loss={test_stats['val_loss']:.4f}, Test PPL={test_stats['val_ppl']:.2f}")
# ---- Quick generation sanity check ----
with torch.no_grad():
    for pixel_values, ids_loader, paths, raw_labels in test_loader:
        pixel_values = pixel_values.to(device)
        gen_ids = model.generate(
            pixel_values=pixel_values,
            bos_id=bos_id, eos_id=eos_id,
            max_new_tokens=256, top_p=0.9, temperature=0.9, greedy=True
        )
        print("Predictions (first batch):")
        for i in range(min(gen_ids.size(0), 8)):
            print(f"\nGEN {i+1}:", tokenizer.decode(gen_ids[i].tolist()))
            print(f"TGT {i+1}:", tokenizer.decode(ids_loader[i].tolist()))
            # Calculate loss between generated and target sequences
        del pixel_values, ids_loader, paths, raw_labels, gen_ids
        torch.cuda.empty_cache()
        break

Evaluating: 100%|██████████| 10/10 [00:09<00:00,  1.01it/s]


Epoch 1: Train Loss=3.0362, PPL=20.97 | Val Loss=3.1141, Val PPL=22.79


Evaluating: 100%|██████████| 10/10 [00:10<00:00,  1.01s/it]


Epoch 2: Train Loss=3.1639, PPL=23.78 | Val Loss=3.0976, Val PPL=22.41


Evaluating: 100%|██████████| 10/10 [00:10<00:00,  1.01s/it]


...

Epoch 49: Train Loss=2.8331, PPL=17.39 | Val Loss=2.9334, Val PPL=19.03


Evaluating: 100%|██████████| 10/10 [00:10<00:00,  1.03s/it]


Epoch 50: Train Loss=2.8511, PPL=17.49 | Val Loss=2.9334, Val PPL=19.03


Evaluating: 100%|██████████| 1/1 [00:00<00:00,  1.03it/s]


Test Loss=2.8813, Test PPL=17.84
Predictions (first batch):

GEN 1: 1. pa and lateral views of the chest demonstrate stable appearance of the right upper extremity picc line. 2. the lungs are clear without focal consolidation. 3. no pleural effusion. 4. no acute osseous abnormality. degenerative changes of the thoracic spine. degenerative changes of the spine. degenerative changes of the spine. degenerative changes. degenerative changes of the spine. degenerative changes of the spine. degenerative changes of the spine. spine. spine... degenerative changes.. degenerative changes of the spine. degenerative changes of the spine.. spine degenerative changes of the spine. degenerative changes of the spine.. degenerative changes of the spine. degenerative changes of the spine.. degenerative changes of the spine... spine... of the spine.. spine. spine. degenerative changes of the spine. degenerative changes of the spine., and spine. degenerative changes
TGT 1: 1. chest 2 views, demonstrate no